In [ ]:
import cv2
import glob 
import numpy as np
import Hog_Algo
from sklearn import svm
import pickle
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
#code to recognize alpha numeric characters
#step-1:Reading the training images
images = [cv2.imread(file) for file in glob.glob('C:/Users/Winni/Desktop/final Mini/AlphaNumericRecog/train_Numbers/*.jpg')]
g_images = []


In [ ]:
#step-2:Preprocessing of the image
I=0
for image in images:
    width = 160
    height =160
    
    #step-2(a):Image resize to generalize
    dim = (width, height)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    
    #step-2(b):Coversion to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #step-2(c):Canny Edge Detection
    min_val=np.mean(gray_image)*75/100
    max_val=np.mean(gray_image)*125/100
    canny_image=cv2.Canny(gray_image,min_val,max_val)
    ret, thresh = cv2.threshold(canny_image, 127, 255, 0)
    _, contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    area=[]
    
    for c in contours:
        area.append(cv2.contourArea(c))
    
    maxpos = area.index(max(area))
    big=contours[maxpos]     
    x,y,w,h = cv2.boundingRect(big)
    box_img=cv2.rectangle(canny_image,(x,y),(x+w,y+h),(0,255,0),2)
    img=box_img
    width = 160
    height =160
    dim = (width, height)
    image = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    #step-2(d):Append processed images to g_images
    g_images.append(image)


In [ ]:
#step-3:Recognizing Alphanumeric Characters

#step-3(a):Histogram of gradients (importing functions from Hog_Algo)
No_of_x_partitions = 4
No_of_y_partitions = 4
digits_fv = []

for image in g_images:
    num = []
    #Step-HOG(a): Slicing
    partition = Hog_Algo.partitions(image,No_of_x_partitions,No_of_y_partitions)
    #Step-HOG(b): Filter
    for x in partition:
        p = Hog_Algo.apply_filter(x)
        for x in p:
            num.append(x)
    #Step-HOG(c): Creation of Feature vector
    digits_fv.append(num)
    
#step-3(b): Saving Feature Vector
with open('digits_hog.txt', 'w') as filehandle:  
    for digit in digits_fv:
        for num in digit:
            filehandle.write('%s ' % num)
        filehandle.write('\n')
        


In [ ]:
#step-4: SVM
#svm 
train_data = np.array(digits_fv)
class_label=np.array([ 0,0,0,0,0,0,0,0,0,0,0,
                       1,1,1,1,1,1,1,1,1,1,
                       2,2,2,2,2,2,2,2,2,2,
                       3,3,3,3,3,3,3,3,3,3,
                       4,4,4,4,4,4,4,4,4,4,
                       5,5,5,5,5,5,5,5,5,5,
                       6,6,6,6,6,6,6,6,6,6,
                       7,7,7,7,7,7,7,7,7,7,
                       8,8,8,8,8,8,8,8,8,8,
                       9,9,9,9,9,9,9,9,9,9,9,
                       1,2,3,4,5,6,7,8,9
                      
                      ])

#SVM classification
    
svm_classifier = svm.SVC(kernel = 'linear', C = 1)
svm_classifier.fit(train_data, class_label)        
filename = 'svm_model_num.sav'
pickle.dump(svm_classifier, open(filename, 'wb'))

#KNN classification

knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(train_data, class_label)
filename1 = 'knn_model_num.sav'
pickle.dump(knn_classifier, open(filename1, 'wb'))
cv2.waitKey(0)
cv2.destroyAllWindows()
